In [1]:
import requests
from bs4 import BeautifulSoup
from datetime import date
import pandas as pd
import re
import os


In [2]:
# url='https://gorkhapatraonline.com/categories/national'

# response= requests.get(url)
# soup = BeautifulSoup(response.content, 'html.parser')
# print(soup.prettify())


In [3]:
urls =[
        'https://gorkhapatraonline.com/',
        # 'https://gorkhapatraonline.com/categories/national'
        ]

data = {
        'शीर्षक': [],
        'विवरण': [],
        }
df = pd.DataFrame(data)
df
      

,शीर्षक,विवरण


In [13]:


for url in urls:
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()  # Raises an HTTPError for bad responses
        soup = BeautifulSoup(response.content, 'html.parser')
        
        article = soup.find_all('section', class_='article-section')
        
        links = []
        for a in article:
            try:
                l = a.find_all('a')
                for link_element in l:
                    href = link_element.get('href')
                    if href:  # Only add non-empty hrefs
                        links.append(href)
            except Exception as error:
                print(f"Error extracting links from article section: {error}")
                continue
        # Remove duplicates by converting the list to a set and back to a list
        links = list(set(links))
        # Filter out links that do not contain 'news'
        links = [link for link in links if 'news' in link]  
        # Filter out links that are not HTTP or HTTPS
        links = [link for link in links if link.startswith(('http://', 'https://'))]

        # Print the number of links found
        print(f"Found {len(links)} links in {url}")
        
        for link in links:
            try:
                # Handle relative URLs if needed
                if link.startswith('/'):
                    # Assuming you need to prepend base URL
                    # link = base_url + link  # Uncomment and set base_url if needed
                    pass
                
                response = requests.get(link, timeout=10)
                response.raise_for_status()
                soup = BeautifulSoup(response.content, 'html.parser')
                
                section = soup.find_all('section', class_='article-section')
                
                for sec in section:
                    try:
                        # Check if h1 exists before accessing text
                        h1_element = sec.find('h1')
                        if h1_element is None:
                            print(f"No h1 found in section for link: {link}")
                            continue
                            
                        title = h1_element.text.strip()
                        print(title)
                        
                        des = sec.find_all('div', class_="blog-details")
                        description = ''
                        
                        for desc in des:
                            try:
                                para = desc.find_all('p')
                                for p in para:
                                    text = p.text.strip()
                                    description += text + ' '
                            except Exception as e:
                                print(f"Error extracting paragraph text: {e}")
                                continue
                        
                        # Clean up the description
                        description= '"' + description + '"'
                        
                        # Only add to dataframe if we have both title and description
                        if title and description.strip():
                            df.loc[len(df)] = [title, description]
                        
                    except Exception as e:
                        print(f"Error processing section: {e}")
                        continue
                        
            except requests.exceptions.RequestException as e:
                print(f"Request error for link {link}: {e}")
                continue
            except Exception as e:
                print(f"Unexpected error processing link {link}: {e}")
                continue
    
    except requests.exceptions.RequestException as e:
        print(f"Request error for URL {url}: {e}")
        continue
    except Exception as e:
        print(f"Unexpected error processing URL {url}: {e}")
        continue

# Save to CSV (moved outside the loop to save once after processing all URLs)
try:
    today = date.today().strftime("%Y-%m-%d")
    foldername = '/var/home/ramrshrcg/Desktop/Python/scrapping/News_csv/gorkhapatra'
    
    # Create directory if it doesn't exist
    os.makedirs(foldername, exist_ok=True)
    
    file_path = f"{foldername}/{today}_gorkhapatra_news.csv"
    
    if os.path.exists(file_path):
        df.to_csv(file_path, mode='a', index=False, header=False)
    else:
        df.to_csv(file_path, index=False)
        
    print(f"Data saved to {file_path}")
    
except Exception as e:
    print(f"Error saving to CSV: {e}")

Found 116 links in https://gorkhapatraonline.com/
यस्तो छ आजको सागसब्जी र फलफूलको थोक मूल्य
प्रदेश बजेटका विविधता
‘भुइँमान्छेहरू सांसदबाट विकास चाहन्छन्’
बजेट कार्यान्वयन उदाहरणीय हुने
दुई महिनामा ४७ फरार अभियुक्त पक्राउ
चोभार गल्छीमा गेग्रान, बाढी थुनिने जोखिम
६० किलो सुन तस्कर गर्ने अमला रोका पक्राउ
अन्धविश्वासविरुद्धको आवाज हो ‘मोहर’ : निर्देशक ओम प्रतीक
सगरमाथा हिमालको दिगोपन
राजस्व र व्ययको वार्षिक अनुमान माथिको छलफल समाप्त
बिचौलियाको सिन्डिकेट जनप्रतिनिधि संस्थाभन्दा बलियो छः विराजभक्त श्रेष्ठ
म्याग्दीमा हुने गण्डकी प्रदेशस्तरीय भलिबलको तयारी अन्तिम
पशुपतिमा मनाइयो त्रिशूल जात्रा
बुद्धिचालमा पुरुषोत्तमको एकल अग्रता
कैलालीमा सुत्केरी मृत फेला
जनताको चाहनाअनुसार काम गरिरहेका छौं : मन्त्री गुरुङ
अब एकीकृत र संयुक्त परीक्षा हुन्छ :  वीरबहादुर राई
नगरपालिकाको मुद्दामै सिद्धियो चौबिस लाख
जनता मालिक हुन सकेनन् : सांसद प्रभु साह
प्रधानमन्त्रीसहित तीन दलका शीर्ष नेताको बैठक सिंहदरबारमा
सूचना प्रविधिको विकास तीन तहको समन्वयमा
पुनः प्रयोग योग्य रकेटको सफल परीक्षण
सिरिज जित्ने लक्ष्यका साथ न

In [6]:
df

,शीर्षक,विवरण
0,यस्तो छ आजको सागसब्जी र फलफूलको थोक मूल्य,"काठमाडौँ, असार६ गते । कालीमाटी फलफूल तथा तरक..."
1,प्रदेश बजेटका विविधता,लामो एकात्मक शासनव्यवस्थामा सन्तुलित विकास हुन...
2,‘भुइँमान्छेहरू सांसदबाट विकास चाहन्छन्’,"काठमाडौँ, जेठ १७ गते । प्रतिनिधिसभा सदस्य गङ्ग..."
3,बजेट कार्यान्वयन उदाहरणीय हुने,"गोरखापत्र समाचारदाता\nकाठमाडौँ, असार ६ गते । उ..."
4,दुई महिनामा ४७ फरार अभियुक्त पक्राउ,"सिराज खाननेपालगन्ज, असार ६ गते । जिल्ला प्रहरी..."
...,...,...
105,कर्मथलोमा रसिकको सम्झना,"पशुपतिनगर समाचारदाता\nपशुपतिनगर, असार ४ गते । ..."
106,दैलेखमा ११२ अर्ब घनमिटर प्राकृतिक ग्यास रहेको ...,"लक्ष्मी सापकोटा/यामराज विष्ट\nकाठमाडौँ/दैलेख, ..."
107,"विनयी खोलामा आएको बाढीले बस बगायो, यात्रु सकुशल","नवलपुर (नवलपरासी), असार ६ गते । पूर्वपश्चिम रा..."
108,"संसदमा प्रतिपक्षीको विरोध, प्रधानमन्त्रीको प्र...","काठमाडौँ, असार ६ गते । प्रतिनिधिसभाको शुक्रबार..."
